# Machine Learning

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
import os
from sklearn.linear_model import LogisticRegression

In [2]:
df_train=pd.read_csv("data/application_train.csv",encoding="utf-8",index_col=0)
df_test=pd.read_csv("data/application_test.csv",encoding="utf-8",index_col=0)
df = pd.concat([df_train,df_test])
df=df.dropna()
df

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100083,0.0,Cash loans,M,Y,Y,0,103500.0,573628.5,24435.0,463500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100145,0.0,Cash loans,F,Y,Y,1,202500.0,260725.5,16789.5,198000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100179,0.0,Cash loans,F,Y,N,0,202500.0,675000.0,53329.5,675000.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
100190,0.0,Cash loans,M,Y,N,0,162000.0,263686.5,24781.5,238500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100295,1.0,Cash loans,M,Y,N,1,225000.0,1019205.0,31032.0,774000.0,...,1,0,0,0,0.0,0.0,0.0,6.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456083,0.0,Cash loans,F,Y,Y,2,112500.0,361462.5,16051.5,274500.0,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0
456084,0.0,Cash loans,F,Y,Y,1,99000.0,675000.0,21906.0,675000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
456140,1.0,Cash loans,F,Y,Y,1,261000.0,711454.5,47673.0,643500.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0


In [3]:
colonnes=[colonne for colonne in df]
types_colonnes=list(map(lambda x : {x : list(set(df[x].map(lambda x : str(type(x)))))} , colonnes))

In [4]:
colonnes_str=list(filter(lambda x : x[list(x)[0]]==["<class 'str'>"], types_colonnes))

In [5]:
def vectorisation(col):
    voc=list(set(list(col)))
    val={voc[i]:i for i in range(len(voc))}
    return col.map(lambda x : val[x])



for i in [list(i)[0] for i in colonnes_str]:
    df[i]=vectorisation(df[i])

In [6]:
df

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100083,0.0,0,0,0,1,0,103500.0,573628.5,24435.0,463500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100145,0.0,0,1,0,1,1,202500.0,260725.5,16789.5,198000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100179,0.0,0,1,0,0,0,202500.0,675000.0,53329.5,675000.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
100190,0.0,0,0,0,0,0,162000.0,263686.5,24781.5,238500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100295,1.0,0,0,0,0,1,225000.0,1019205.0,31032.0,774000.0,...,1,0,0,0,0.0,0.0,0.0,6.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456083,0.0,0,1,0,1,2,112500.0,361462.5,16051.5,274500.0,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0
456084,0.0,0,1,0,1,1,99000.0,675000.0,21906.0,675000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
456140,1.0,0,1,0,1,1,261000.0,711454.5,47673.0,643500.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0


In [7]:
type(type(''))

type

In [8]:
set(df_train['FONDKAPREMONT_MODE'].fillna("").map(type))

{str}

In [9]:
df.corr()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
TARGET,1.000000,-0.045993,-0.033103,NaN,0.003398,-0.019993,-0.039762,-0.014634,-0.000765,-0.021950,...,-0.006139,0.013976,-0.008259,-0.003892,0.012156,0.014616,0.015000,-0.004202,0.016465,0.033832
NAME_CONTRACT_TYPE,-0.045993,1.000000,-0.000923,NaN,0.078037,0.020854,-0.030260,-0.248492,-0.271951,-0.212679,...,-0.020273,-0.008457,-0.011349,0.043487,-0.007084,-0.013953,-0.017467,-0.019016,-0.022528,-0.063929
CODE_GENDER,-0.033103,-0.000923,1.000000,NaN,0.056212,0.020016,-0.104820,0.033866,-0.006283,0.033341,...,-0.000468,0.014921,0.018585,-0.014969,-0.013602,-0.004925,0.005709,-0.016955,-0.001697,-0.017062
FLAG_OWN_CAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FLAG_OWN_REALTY,0.003398,0.078037,0.056212,NaN,1.000000,0.029913,-0.005579,-0.041779,0.007923,-0.048085,...,-0.096187,-0.025383,-0.040270,-0.022632,0.018160,-0.013311,-0.004906,-0.009066,0.030353,0.068681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AMT_REQ_CREDIT_BUREAU_DAY,0.014616,-0.013953,-0.004925,NaN,-0.013311,-0.002712,-0.001641,0.018995,0.014015,0.019636,...,0.005988,-0.001495,-0.002006,-0.000945,0.214817,1.000000,0.215462,-0.002107,-0.014587,0.003018
AMT_REQ_CREDIT_BUREAU_WEEK,0.015000,-0.017467,0.005709,NaN,-0.004906,-0.001043,0.024093,0.012926,0.026397,0.014701,...,-0.007608,-0.004512,-0.006055,-0.002853,0.021885,0.215462,1.000000,-0.012199,-0.043595,0.024866
AMT_REQ_CREDIT_BUREAU_MON,-0.004202,-0.019016,-0.016955,NaN,-0.009066,-0.023803,0.024042,0.056718,0.025826,0.053417,...,-0.009728,-0.003885,-0.004528,-0.005364,0.013652,-0.002107,-0.012199,1.000000,-0.035314,-0.010767
AMT_REQ_CREDIT_BUREAU_QRT,0.016465,-0.022528,-0.001697,NaN,0.030353,-0.022028,0.006707,0.003209,0.016595,0.004426,...,-0.006256,-0.010430,-0.013997,-0.006595,0.000241,-0.014587,-0.043595,-0.035314,1.000000,0.088951


In [10]:
sorted(list(df.corr()['TARGET']))

[-0.1507200772848682,
 -0.1309687926621415,
 -0.12954752718226564,
 -0.049369031988697916,
 -0.046110202090804436,
 -0.045992703650885466,
 -0.04599007525117904,
 -0.04520611702482206,
 -0.039761625493229244,
 -0.03331296880350878,
 -0.03310334034748959,
 -0.03290405336807689,
 -0.031688266859563295,
 -0.03116154790316484,
 -0.029713532258080762,
 -0.028102952042565483,
 -0.028068889862141462,
 -0.027218252805351836,
 -0.02692248968131364,
 -0.026595365458070327,
 -0.026565709031903546,
 -0.02649079593614602,
 -0.025762022546263752,
 -0.02554521272133045,
 -0.0251158578648942,
 -0.024612157911699083,
 -0.0244225774045672,
 -0.024361055249211727,
 -0.02420732926810737,
 -0.02343507830870301,
 -0.023181578957477286,
 -0.022802532584821233,
 -0.02244149784412661,
 -0.022205024252245114,
 -0.02194979271485281,
 -0.021425035167508908,
 -0.021299647781074346,
 -0.02122539112563392,
 -0.019992588090237167,
 -0.018974830538604232,
 -0.018369065009061165,
 -0.017046478961749024,
 -0.01544742727

In [11]:
df

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100083,0.0,0,0,0,1,0,103500.0,573628.5,24435.0,463500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100145,0.0,0,1,0,1,1,202500.0,260725.5,16789.5,198000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100179,0.0,0,1,0,0,0,202500.0,675000.0,53329.5,675000.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
100190,0.0,0,0,0,0,0,162000.0,263686.5,24781.5,238500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100295,1.0,0,0,0,0,1,225000.0,1019205.0,31032.0,774000.0,...,1,0,0,0,0.0,0.0,0.0,6.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456083,0.0,0,1,0,1,2,112500.0,361462.5,16051.5,274500.0,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0
456084,0.0,0,1,0,1,1,99000.0,675000.0,21906.0,675000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
456140,1.0,0,1,0,1,1,261000.0,711454.5,47673.0,643500.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0


In [12]:
y_ech=np.array(list(df['TARGET']))
y_ech.shape

(8602,)

In [13]:
x_ech=np.array(list(zip([list(df[i]) for i in df if i!='TARGET']))).reshape(-1,120)
x_ech.shape

(8602, 120)

In [14]:
train_x, test_x, train_y, test_y  = train_test_split(x_ech,y_ech, test_size = 0.25, random_state = 44)

In [16]:
line=LinearSVC()
line.fit(train_x,train_y)
print("La précision de LinearSVC est : {}%".format(round(line.score(test_x,test_y)*100,2)))


foret = RandomForestClassifier(n_estimators=750)
foret.fit(train_x,train_y)
print("La précision de RandomForestClassifier est : {}%".format(round(foret.score(test_x,test_y)*100,2)))


gradient=GradientBoostingClassifier()
gradient.fit(train_x,train_y)
print("La précision de GradientBoostingClassifier est : {}%".format(round(foret.score(test_x,test_y)*100,2)))


lr = LogisticRegression()
lr.fit(train_x,train_y)
print("La précision de LogisticRegression est : {}%".format(round(lr.score(test_x,test_y)*100,2)))

C:\Users\minim\Anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



La précision de LinearSVC est : 89.82%
La précision de RandomForestClassifier est : 94.33%
La précision de GradientBoostingClassifier est : 94.33%
La précision de LogisticRegression est : 65.09%


C:\Users\minim\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

